In [8]:
from customdataset import CustomDataset
from SSD import *
from train_step import train_step
from eval_step import test_step
import os
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import time

In [3]:
PATH=os.getenv("HOME")+"/aiffel/ECG_data/physionet.org/files/circor-heart-sound/1.0.3/training_data"
file_list = os.listdir(PATH)
txt_list = [os.path.join(PATH, file) for file in file_list if file.endswith(".txt")]

# 환자 아이디를 훈련, 검증, 테스트 데이터셋으로 나눔
train_patient_txt, extra_patient_txt = train_test_split(txt_list, test_size=0.4, random_state=42)
valid_patient_txt, test_patient_txt = train_test_split(extra_patient_txt, test_size=0.5, random_state=42)

print(len(txt_list) ,len(train_patient_txt),
      len(valid_patient_txt) ,len(test_patient_txt)
     )

942 565 188 189


In [4]:
def my_collate_fn(batch):
    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0])  # sample[0]은 화상 gt
        targets.append(torch.FloatTensor(sample[1]))  # sample[1]은 어노테이션 gt

    imgs = torch.stack(imgs, dim=0)
    return imgs, targets
BATCHSIZE = 8

In [6]:
s_t=time.time()
dataset = CustomDataset(PATH, train_patient_txt, target_size=(300, 300), th=5, resizing=True)
train_dataloader = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=True, collate_fn=my_collate_fn)
e_t=time.time()

print(e_t-s_t)

In [9]:
s_t=time.time()
dataset = CustomDataset(PATH, valid_patient_txt, target_size=(300, 300), th=5, resizing=True)
valid_dataloader = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=True, collate_fn=my_collate_fn)
e_t=time.time()

print(e_t-s_t)

42.460023403167725


In [ ]:
s_t=time.time()
dataset = CustomDataset(PATH, test_patient_txt, target_size=(300, 300), th=5, resizing=True)
test_patient_txt = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=True, collate_fn=my_collate_fn)
e_t=time.time()

print(e_t-s_t)

# train_step

In [10]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = build_model("Train", input_channels=1, is_freeze=False)
test_model =  build_model("Test", input_channels=1)
#model_weight_path='./objectdetection_model/ssd300_weight_100.pth'
#weight = torch.load(model_weight_path)
#model.load_state_dict(weight)

In [11]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
_optim_type = "Adam"
_lr=2e-3
_model_name = "MnetSSD"
_is_freeze = "False"

train_step(model, test_model, train_dataloader, valid_dataloader, epoch_num = 100, batchsize=BATCHSIZE,
           optim_type = _optim_type, lr = _lr, device = DEVICE, model_name = _model_name,
           is_wandb=False, is_freeze=_is_freeze)


Already './objectdetection_model' DIR path
Epoch : 1 / 100
Current Batch 0 / 755 | Cls Loss : 2.261, Loc Loss : 3.436, Total Loss : 5.697 | 50 iter time 0.1002: 
Current Batch 50 / 755 | Cls Loss : 2.244, Loc Loss : 3.503, Total Loss : 5.747 | 50 iter time 4.3800: 
Current Batch 100 / 755 | Cls Loss : 1.918, Loc Loss : 3.147, Total Loss : 5.065 | 50 iter time 4.3158: 
Current Batch 150 / 755 | Cls Loss : 2.124, Loc Loss : 2.874, Total Loss : 4.999 | 50 iter time 4.3777: 
Current Batch 200 / 755 | Cls Loss : 1.642, Loc Loss : 2.896, Total Loss : 4.538 | 50 iter time 4.2691: 
Current Batch 250 / 755 | Cls Loss : 1.271, Loc Loss : 2.827, Total Loss : 4.098 | 50 iter time 4.3983: 
Current Batch 300 / 755 | Cls Loss : 1.254, Loc Loss : 2.781, Total Loss : 4.035 | 50 iter time 4.3006: 
Current Batch 350 / 755 | Cls Loss : 1.250, Loc Loss : 2.832, Total Loss : 4.082 | 50 iter time 4.2971: 
Current Batch 400 / 755 | Cls Loss : 1.615, Loc Loss : 2.724, Total Loss : 4.340 | 50 iter time 4.3877: 

# test_step

In [12]:
_optim_type = "Adam"
_lr = 2e-3
_model_name = "MnetSSD"
_is_freeze = "False"


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = build_model("Test", input_channels=1)
model_weight_path='./objectdetection_model/'+f'{_model_name}_weight_30_{BATCHSIZE}_{_optim_type}_{_is_freeze}.pth'
weight = torch.load(model_weight_path)
model.load_state_dict(weight)
model.eval()

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
s = time.time()
recall, precison, mAP = test_step(model, valid_dataloader, device = DEVICE)
e = time.time()
print(e-s)

7.8421547412872314


In [14]:
print(recall, precison, mAP)

0.4574432837995387 0.6236401414493017 0.3694359239464318


In [7]:
!pip freeze | grep -E "torch" >> requirements.txt
!pip freeze | grep -E "skimage" >> requirements.txt
!pip freeze | grep -E "numpy" >> requirements.txt
!pip freeze | grep -E "librosa" >> requirements.txt
!pip freeze | grep -E "wandb" >> requirements.txt
!pip freeze | grep -E "scipy" >> requirements.txt
!pip freeze | grep -E "time" >> requirements.txt
!pip freeze | grep -E "pandas" >> requirements.txt
!pip freeze | grep -E "matplotlib" >> requirements.txt

In [7]:
!pip freeze | grep -E "numpy" >> requirements.txt